# Initialisation steps

## Importing required modules

In [1]:
import pandas as pd
from drg_connect import Snowflake
import qgrid 
from datetime import timedelta, datetime
import math

import warnings
warnings.filterwarnings('ignore')

In [2]:
##defining parameters of snowflake
snow = Snowflake(role = 'RWD_ANALYTICS_RW',database='SANDBOX_ANALYTICS',schema = 'SANDBOX')
engine = snow.engine

%reload_ext sql_magic
%config SQL.output_result = True  #Enable output to std out
%config SQL.notify_result = False #disable browser notifications
%config SQL.conn_name = 'engine'  #Set the sql_magic connection engine

## ICD codes tables

In [4]:
%%read_sql

select
    *
from
    RWD_DB.RWD.ICD_GROUPER
where
    (level_4_long_description_icd10 ilike '%spinal%'
    and level_4_long_description_icd10 ilike '%cord%'
    and level_4_long_description_icd10 ilike '%injury%')

Query started at 03:13:09 PM India Standard Time; Query executed in 0.09 m

,id,level_1,level_1_description,level_2,level_2_description,level_3,level_3_description,level_4,icd9_mapped_codes,icd9_description,level_4_short_description_icd10,level_4_long_description_icd10,create_ts,update_ts
0,27188,P00 - P969,Certain conditions originating in the perinata...,P10-P15,Birth trauma,P11-P119,Other birth injuries to central nervous system,P115,7674,Injury to spine and spinal cord due to birth t...,Birth injury to spine and spinal cord,Birth injury to spine and spinal cord,2017-11-02,2017-11-02
1,32824,S00 - T889XXS,Injury poisoning and certain other consequence...,S10-S19,Injuries to the neck,S14-S149,Injury of nerves and spinal cord at neck level,S14,No map,No map,Injury of nerves and spinal cord at neck level,Injury of nerves and spinal cord at neck level,2017-11-02,2017-11-02
2,32830,S00 - T889XXS,Injury poisoning and certain other consequence...,S10-S19,Injuries to the neck,S14-S149,Injury of nerves and spinal cord at neck level,S1410,No map,No map,Unspecified injury of cervical spinal cord,Unspecified injury of cervical spinal cord,2017-11-02,2017-11-02
3,32831,S00 - T889XXS,Injury poisoning and certain other consequence...,S10-S19,Injuries to the neck,S14-S149,Injury of nerves and spinal cord at neck level,S14101,No map,No map,Unspecified injury at C1 level of cervical spi...,Unspecified injury at C1 level of cervical spi...,2017-11-02,2017-11-02
4,32832,S00 - T889XXS,Injury poisoning and certain other consequence...,S10-S19,Injuries to the neck,S14-S149,Injury of nerves and spinal cord at neck level,S14101A,80600,Closed fracture of C1-C4 level with unspecifie...,Unsp injury at C1 level of cervical spinal cor...,Unspecified injury at C1 level of cervical spi...,2017-11-02,2017-11-02
5,32833,S00 - T889XXS,Injury poisoning and certain other consequence...,S10-S19,Injuries to the neck,S14-S149,Injury of nerves and spinal cord at neck level,S14101D,No map,No map,Unsp injury at C1 level of cervical spinal cor...,Unspecified injury at C1 level of cervical spi...,2017-11-02,2017-11-02
6,32834,S00 - T889XXS,Injury poisoning and certain other consequence...,S10-S19,Injuries to the neck,S14-S149,Injury of nerves and spinal cord at neck level,S14101S,No map,No map,Unsp injury at C1 level of cervical spinal cor...,Unspecified injury at C1 level of cervical spi...,2017-11-02,2017-11-02
7,32835,S00 - T889XXS,Injury poisoning and certain other consequence...,S10-S19,Injuries to the neck,S14-S149,Injury of nerves and spinal cord at neck level,S14102,No map,No map,Unspecified injury at C2 level of cervical spi...,Unspecified injury at C2 level of cervical spi...,2017-11-02,2017-11-02
8,32836,S00 - T889XXS,Injury poisoning and certain other consequence...,S10-S19,Injuries to the neck,S14-S149,Injury of nerves and spinal cord at neck level,S14102A,80600,Closed fracture of C1-C4 level with unspecifie...,Unsp injury at C2 level of cervical spinal cor...,Unspecified injury at C2 level of cervical spi...,2017-11-02,2017-11-02
9,32837,S00 - T889XXS,Injury poisoning and certain other consequence...,S10-S19,Injuries to the neck,S14-S149,Injury of nerves and spinal cord at neck level,S14102D,No map,No map,Unsp injury at C2 level of cervical spinal cor...,Unspecified injury at C2 level of cervical spi...,2017-11-02,2017-11-02


In [10]:
%%read_sql

create or replace temporary table st_spc_ref1 as

select
    'dx' as cat1,
    'spc' cat2,
    null as cat3,
    null as cat4,
    null as cat5,
    null as cat6,
    'icd_10' as type,
    level_4 as value,
    level_4_long_description_icd10 as description,
    current_date() as active_date,
    'Ravdeep' as active_reason,
    null as deactive_date,
    null as deactive_reason,
    icd9_mapped_codes as icd9_map,
    icd9_description as icd9_details,    
    'RWD_DB.RWD.ICD_GROUPER' as source
from 
    RWD_DB.RWD.ICD_GROUPER
where
    ((level_4_long_description_icd10 ilike '%spinal%'
    and level_4_long_description_icd10 ilike '%cord%'
    and level_4_long_description_icd10 ilike '%injury%')
     or (level_4_long_description_icd10 ilike '%cord%'
    and level_4_long_description_icd10 ilike '%syndrome%'))

Query started at 04:09:41 PM India Standard Time; Query executed in 0.08 m

,status
0,Table ST_SPC_REF1 successfully created.


In [11]:
snow.select("select * from st_spc_ref1").to_excel("out/spc_check.xlsx", index=False)

In [9]:
# Export to check, and then will inactivate codes that are not relevant and re-upload as final reference table
df.to_excel("out/spc_check.xlsx", index=False)

NameError: name 'df' is not defined